<a href="https://colab.research.google.com/github/Shrutika-TechSavvy/Google-Colab-Codes/blob/main/Vector_Multiplication_MPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y mpich



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37
Suggested packages:
  mpich-doc
The following NEW packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37 mpich
0 upgraded, 5 newly installed, 0 to remove and 41 not upgraded.
Need to get 14.2 MB of archives.
After this operation, 102 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libslurm37 amd64 21.08.5-2ubuntu1 [542 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 hwloc-nox amd64 2.7.0-2ubuntu1 [205 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich12 amd64 4.0-3 [5,866 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mpich amd64 4.0-3 [197 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich-dev amd64 4.0-3 [7,375 kB]
Fetched 14.2 MB in 

In [2]:
%%writefile mpi_dynamic.c
#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>

int main(int argc, char* argv[]) {
    int rank, size, n;

    // Initialize MPI Environment
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    //  Rank 0 takes input from user
    if (rank == 0) {
        printf("\nEnter size of matrix (n x n) : ");
        scanf("%d", &n);

        //  Edge case: size must be >= number of processes
        if (n < size) {
            printf("\n❌ Number of rows must be >= number of processes!\n");
        }
    }

    //  Broadcast size to all processes
    MPI_Bcast(&n, 1, MPI_INT, 0, MPI_COMM_WORLD);

    if (n < size) {
        MPI_Finalize();
        return 0;
    }

    // Each process gets n/size rows
    int rows_per_proc = n / size;

    //  Dynamic allocation
    int *matrix = NULL;
    int *vector = (int*)malloc(n * sizeof(int));
    int *local_matrix = (int*)malloc(rows_per_proc * n * sizeof(int));
    int *local_result = (int*)malloc(rows_per_proc * sizeof(int));
    int *final_result = NULL;

    //  Initialize vector (same in all processes)
    for (int i = 0; i < n; i++)
        vector[i] = i + 1; // 1,2,3,...

    //  Only Rank 0 initializes full matrix
    if (rank == 0) {
        matrix = (int*)malloc(n * n * sizeof(int));
        printf("\nInput Matrix:\n");
        for (int i = 0; i < n; i++) {
            for (int j = 0; j < n; j++) {
                matrix[i*n + j] = (i+1) + (j+1); // simple pattern
                printf("%d ", matrix[i*n + j]);
            }
            printf("\n");
        }

        printf("\nNumber of Processes Running in Parallel: %d\n", size);
    }

    //  Scatter rows among processes
    MPI_Scatter(matrix,
                rows_per_proc * n,
                MPI_INT,
                local_matrix,
                rows_per_proc * n,
                MPI_INT,
                0,
                MPI_COMM_WORLD);

    //  Multiply assigned rows with vector
    for (int i = 0; i < rows_per_proc; i++) {
        local_result[i] = 0;
        for (int j = 0; j < n; j++) {
            local_result[i] += local_matrix[i*n + j] * vector[j];
        }
    }

    //  Gather results on root
    if (rank == 0)
        final_result = (int*)malloc(n * sizeof(int));

    MPI_Gather(local_result,
               rows_per_proc,
               MPI_INT,
               final_result,
               rows_per_proc,
               MPI_INT,
               0,
               MPI_COMM_WORLD);

    //  Print final result on root
    if (rank == 0) {
        printf("\nFinal Output Vector (Matrix × Vector):\n");
        for (int i = 0; i < n; i++)
            printf("%d ", final_result[i]);
        printf("\n\n");
    }

    //  Free dynamically allocated memory
    free(vector);
    free(local_matrix);
    free(local_result);
    if (rank == 0) {
        free(matrix);
        free(final_result);
    }

    MPI_Finalize();
    return 0;
}



Writing mpi_dynamic.c


In [3]:
!mpicc mpi_dynamic.c -o mpi_dynamic


In [5]:
!mpirun -np 4 ./mpi_dynamic


--------------------------------------------------------------------------
mpirun has detected an attempt to run as root.

Running as root is *strongly* discouraged as any mistake (e.g., in
defining TMPDIR) or bug can result in catastrophic damage to the OS
file system, leaving your system in an unusable state.

We strongly suggest that you run mpirun as a non-root user.

You can override this protection by adding the --allow-run-as-root option
to the cmd line or by setting two environment variables in the following way:
the variable OMPI_ALLOW_RUN_AS_ROOT=1 to indicate the desire to override this
protection, and OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 to confirm the choice and
add one more layer of certainty that you want to do so.
We reiterate our advice against doing so - please proceed at your own risk.
--------------------------------------------------------------------------
